In [67]:
from gensim.models import word2vec
#import log_config
import os
import codecs
import sys
import json
import keras
import numpy as np
from gensim.utils import simple_preprocess
from keras.engine import Input
from keras.layers import Embedding, merge, Reshape
from keras.models import Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.pooling import GlobalMaxPooling1D

model = word2vec.Word2Vec.load('./character.model')

In [ ]:
def load_embeddings(embeddings_path, vocab_path, **params):
    model = word2vec.Word2Vec.load('./character.model')
    weights = model.wv.syn0
    np.save(open(embeddings_path, 'wb'), weights)
    vocab = dict([(k, v.index) for k, v in model.wv.vocab.items()])
    with open(vocab_path, 'w') as f:
        f.write(json.dumps(vocab))
    
    return model

# tokenizer: can change this as needed
tokenize = lambda x: simple_preprocess(x)


def create_embeddings(data_dir,
                      embeddings_path='embeddings.npz',
                      vocab_path='map.json',
                      **params):
    """
    Generate embeddings from a batch of text
    :param embeddings_path: where to save the embeddings
    :param vocab_path: where to save the word-index map
    """

    class SentenceGenerator(object):
        def __init__(self, dirname):
            self.dirname = dirname

        def __iter__(self):
            for fname in os.listdir(self.dirname):
                for line in open(os.path.join(self.dirname, fname)):
                    yield tokenize(line)

    sentences = SentenceGenerator(data_dir)

    model = Word2Vec(sentences, **params)
    weights = model.wv.smyn0
    np.save(open(embeddings_path, 'wb'), weights)

    vocab = dict([(k, v.index) for k, v in model.wv.vocab.items()])
    with open(vocab_path, 'w') as f:
        f.write(json.dumps(vocab))


def load_vocab(vocab_path='map.json'):
    """
    Load word -> index and index -> word mappings
    :param vocab_path: where the word-index map is saved
    :return: word2idx, idx2word
    """

    with open(vocab_path, 'r') as f:
        data = json.loads(f.read())
    word2idx = data
    idx2word = dict([(v, k) for k, v in data.items()])
    return word2idx, idx2word


def word2vec_embedding_layer(embeddings_path='embeddings.npz'):
    """
    Generate an embedding layer word2vec embeddings
    :param embeddings_path: where the embeddings are saved (as a numpy file)
    :return: the generated embedding layer
    """
    
    weights = np.load(open(embeddings_path, 'rb'))
    layer = Embedding(input_dim=weights.shape[0],
                      output_dim=weights.shape[1],
                      weights=[weights])
    return layer



STNCLEN = 30
weights = model.wv.syn0
WVLEN=weights.shape[1]

In [2]:
from IPython.display import Image
Image(filename='cnn.jpeg') 

In [23]:
if __name__ == '__main__':
    load_embeddings(embeddings_path='embeddings.npz',vocab_path='map.json')

    # variable arguments are passed to gensim's word2vec model


    word2idx, idx2word = load_vocab()

    # cosine similarity model
    input_a = Input(shape=(STNCLEN,), dtype='int32', name='input_a')
    input_b = Input(shape=(STNCLEN,), dtype='int32', name='input_b')
    embeddings = word2vec_embedding_layer()
    #embedding_a = (batch, SENTENCELEN, WVLEN)
    embedding_a = embeddings(input_a)
    embedding_b = embeddings(input_b)
    conv1_a = Conv1D(128, kernel_size=(3), strides=(1), padding='valid', activation='relu')(embedding_a)
    conv1_b = Conv1D(128, kernel_size=(3), strides=(1), padding='valid', activation='relu')(embedding_b)
    
#     similarity = merge([embedding_a, embedding_b],
#                        mode='cos', dot_axes=2)

#     model = Model(input=[input_a, input_b], output=[similarity])
#     model.compile(optimizer='sgd', loss='mse')
    
#     while True:
#         word_a = input('First word: ')
#         if word_a not in word2idx:
#             print('Word "%s" is not in the index' % word_a)
#             continue
#         word_b = input('Second word: ')
#         if word_b not in word2idx:
#             print('Word "%s" is not in the index' % word_b)
#             continue
#         output = model.predict([np.asarray([word2idx[word_a]]),
#                                 np.asarray([word2idx[word_b]])])
#         print(output)

In [50]:
from keras import backend as K
class MyPooling(GlobalMaxPooling1D):
    """Global max pooling operation for temporal data.
    # Input shape
        3D tensor with shape: `(batch_size, steps, features)`.
    # Output shape
        2D tensor with shape:
        `(batch_size, features)`
    """

    def call(self, inputs):
        return K.max(inputs, axis=2)

In [53]:
pool1_a = MyPooling()(conv1_a)
pool1_b = MyPooling()(conv1_b)

In [54]:
pool1_a.get_shape

<bound method Tensor.get_shape of <tf.Tensor 'my_pooling_3/Max:0' shape=(?, 28) dtype=float32>>

In [57]:
output = keras.layers.concatenate([pool1_a, pool1_b])

In [58]:
output.shape


TensorShape([Dimension(None), Dimension(56)])

In [64]:
similarity = merge([pool1_a, pool1_b], mode='cos', dot_axes=1)

d:\programdata\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  """Entry point for launching an IPython kernel.
d:\programdata\anaconda3\envs\tensorflow\lib\site-packages\keras\legacy\layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [69]:
similarity = Reshape((1,))(similarity)

In [70]:
similarity.shape

TensorShape([Dimension(None), Dimension(1)])

In [71]:
model = Model(input=[input_a, input_b], output=[similarity])

d:\programdata\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=[<tf.Tenso...)`
  """Entry point for launching an IPython kernel.


In [72]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#model.fit(X_train, y_train_ohe, validation_data=(X_test, y_test_ohe), epochs=20, batch_size=128)

#####TODO
